Lambda School Data Science

*Unit 4, Sprint 1, Module 3*

---

# Document Classification (Prepare)

Today's guided module project will be different. You already know how to do classification. You ready know how to extract features from documents. So? That means you're ready to combine and practice those skills in a kaggle competition. We we will open with a five minute sprint explaining the competition, and then give you 25 minutes to work. After those twenty five minutes are up, I will give a 5-minute demo an NLP technique that will help you with document classification (*and **maybe** the competition*).

Today's all about having fun and practicing your skills. The competition will begin

## Learning Objectives
* <a href="#p0">Part 0</a>: Kaggle Competition
* <a href="#p1">Part 1</a>: Text Feature Extraction & Classification Pipelines
* <a href="#p2">Part 2</a>: Latent Semantic Indexing
* <a href="#p3">Part 3</a>: Word Embeddings with Spacy

# Text Feature Extraction & Classification Pieplines (Learn)
<a id="p1"></a>

## Overview

Sklearn pipelines allow you to stitch together multiple components of a machine learning process. The idea is that you can pass you raw data and get predictions out of the pipeline. This ability to pass raw input and receive a prediction from a singular class makes pipelines well suited for production, because you can pickle a a pipeline without worry about other data preprocessing steps. 

In [2]:
# Import Statements
from sklearn.pipeline import Pipeline
from sklearn.datasets import fetch_20newsgroups
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
# Dataset
from sklearn.datasets import fetch_20newsgroups

categories = ['alt.atheism',
              'talk.religion.misc']

data = fetch_20newsgroups(subset='train', categories=categories)

In [ ]:
#print out data sample

In [2]:
# Create Pipeline Components

vect = TfidfVectorizer(stop_words='english', ngram_range=(1,2))
rfc = RandomForestClassifier()

In [8]:
# Define the Pipeline
pipe = Pipeline([
                 #Vectorizer
                 ('vect', vect), 
                 # Classifier
                 ('clf', rfc)
                ])

# The pipeline puts together a bunch fit then transform,fit them predict.

In [9]:
parameters = {
    'vect__max_df': ( 0.75, 1.0),
    'vect__min_df': (.02, .05),
    'vect__max_features': (500,1000),
    'clf__n_estimators':(5, 10,),
    'clf__max_depth':(15,20)
}

grid_search = GridSearchCV(pipe,parameters, cv=5, n_jobs=-1, verbose=1)
grid_search.fit(data.data, data.target)

Fitting 5 folds for each of 32 candidates, totalling 160 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:    5.1s
[Parallel(n_jobs=-1)]: Done 160 out of 160 | elapsed:   24.3s finished


GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=Pipeline(memory=None,
     steps=[('vect', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 2), norm='l2', preprocessor=None, smooth_idf=True,
...obs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False))]),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid={'vect__max_df': (0.75, 1.0), 'vect__min_df': (0.02, 0.05), 'vect__max_features': (500, 1000), 'clf__n_estimators': (5, 10), 'clf__max_depth': (15, 20)},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=1)

In [10]:
grid_search.best_score_

0.8891481913652275

In [11]:
grid_search.predict(['Send me lots of money now', 'you won the lottery in Nigeria'])

array([1, 1], dtype=int64)

## Follow Along 

What you should be doing now:
1. Join the Kaggle Competition
2. Download the data
3. Train a model (try using the pipe method I just demoed)

### Load Competition Data

In [3]:
import pandas as pd
train = pd.read_csv('./data/ds8-which-whiskey/train.csv')
test = pd.read_csv('./data/ds8-which-whiskey/test.csv')

In [20]:
train.head()

,id,description,category
0,1,A marriage of 13 and 18 year old bourbons. A m...,2
1,2,There have been some legendary Bowmores from t...,1
2,3,This bottling celebrates master distiller Park...,2
3,4,What impresses me most is how this whisky evol...,1
4,9,"A caramel-laden fruit bouquet, followed by une...",2


In [22]:
test.head()

,id,description
0,955,"Think carnival aromas—the good ones, anyway—me..."
1,3532,"A blend of three bourbons, between 6 and 12 ye..."
2,1390,"The nose is focused on cereal, hints of fresh ..."
3,1024,Swiss-based Chapter 7 released this 19 year ol...
4,1902,Valkyrie replaces the current Dark Origins exp...


In [4]:
X_train = train['description']
y_train = train['category']
X_test = test['description']

### Define Pipeline Components

In [30]:
vect = TfidfVectorizer(stop_words='english', ngram_range=(1,2))
model = RandomForestClassifier(n_estimators=200)

pipe = Pipeline([('vect', vect), ('model', model)])

### Define Your Search Space
You're looking for both the best hyperparameters of your vectorizer and your classification model. 

In [31]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform

parameters = {
    'vect__max_df': (0.7, 1.0),
    'vect__min_df': (0.01, 0.1),
    'model__max_depth': (5,10,15,20,None),
    'model__max_features': uniform(0, 1)
}

search = RandomizedSearchCV(pipe,parameters, n_iter=10, cv=5, n_jobs=-1, verbose=1, return_train_score=True)
search.fit(X_train, y_train)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:   20.5s
[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:   36.7s finished


RandomizedSearchCV(cv=5, error_score='raise-deprecating',
          estimator=Pipeline(memory=None,
     steps=[('vect', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 2), norm='l2', preprocessor=None, smooth_idf=True,
...obs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False))]),
          fit_params=None, iid='warn', n_iter=10, n_jobs=-1,
          param_distributions={'vect__max_df': (0.7, 1.0), 'vect__min_df': (0.01, 0.1), 'model__max_depth': (5, 10, 15, 20, None), 'model__max_features': <scipy.stats._distn_infrastructure.rv_frozen object at 0x0000028808EEADA0>},
          pre_dispatch='2*n_jobs', random_state=None, refit=True,
          return_train_score=True, scoring=None, verbose=1)

In [32]:
search.best_score_

0.8859242072699149

### Make a Submission File
*Note:* In a typical Kaggle competition, you are only allowed two submissions a day, so you only submit if you feel you cannot achieve higher test accuracy. For this compeition the max daily submissions are capped at **20**. Submit for each demo and for your assignment. 

In [33]:
# Predictions on test sample
pred = search.predict(X_test)

In [34]:
submission = pd.DataFrame({'id': test['id'], 'category':pred})
submission['category'] = submission['category'].astype('int64')

In [35]:
# Make Sure the Category is an Integer
submission.head()

,id,category
0,955,2
1,3532,2
2,1390,1
3,1024,1
4,1902,1


In [36]:
# Save your Submission File
# Best to Use an Integer or Timestamp for different versions of your model
submission.to_csv('./whiskey-submission-1.csv', index=False)

### Second attempt

In [38]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression()

pipe = Pipeline([('vect', vect), ('model', model)])

parameters = {
    'vect__max_df': ( 0.75, 1.0),
    'vect__min_df': (.02, .05),
    'model__solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']
}

grid_search = GridSearchCV(pipe,parameters, cv=5, n_jobs=-1, verbose=1)
grid_search.fit(X_train, y_train)

Fitting 5 folds for each of 20 candidates, totalling 100 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:    6.3s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:   14.3s finished
C:\Anaconda3\envs\Unit4_Sprint1\lib\site-packages\sklearn\linear_model\logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=Pipeline(memory=None,
     steps=[('vect', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 2), norm='l2', preprocessor=None, smooth_idf=True,
...penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False))]),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid={'vect__max_df': (0.75, 1.0), 'vect__min_df': (0.02, 0.05), 'model__solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=1)

In [39]:
grid_search.best_score_

0.8901778808971385

In [41]:
pred = grid_search.predict(X_test)

In [42]:
submission = pd.DataFrame({'id': test['id'], 'category':pred})
submission['category'] = submission['category'].astype('int64')

In [43]:
submission.head()

,id,category
0,955,2
1,3532,3
2,1390,1
3,1024,1
4,1902,1


In [44]:
submission.to_csv('./whiskey-submission-2.csv', index=False)

In [45]:
# third try

vect = TfidfVectorizer(stop_words='english', ngram_range=(1,2))
model = RandomForestClassifier(n_estimators=200)

pipe = Pipeline([('vect', vect), ('model', model)])

In [52]:
# can't figure out why i'm getting this error message...leaving it for posterity

parameters = {
    'vect__max_df': (0.6, 1),
    'vect__min_df': (0.001, 0.3),
    'model__max_depth': (5,10,15,20,None),
    'model__max_features': uniform(0, 1)
}

search = RandomizedSearchCV(pipe,parameters, n_iter=10, cv=5, n_jobs=-1, verbose=1, return_train_score=True)
search.fit(X_train, y_train)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.


ValueError: max_df corresponds to < documents than min_df

In [47]:
search.best_score_

0.8128383604021655

## Challenge

You're trying to achienve 90% Accuracy on your model.

## Latent Semantic Indexing (Learn)
<a id="p2"></a>

## Overview

In [5]:
# Import

from sklearn.decomposition import TruncatedSVD

svd = TruncatedSVD(n_components=100, # Just here for demo.
                   algorithm='randomized',
                   n_iter=10)

In [10]:
params = { 
    'lsi__svd__n_components': [10,100,250],
    'lsi__vect__max_df':[.9, .95, 1.0],
    'clf__n_estimators':[5,10,20]
}

In [12]:
# LSI
lsi = Pipeline([('vect', vect), ('svd', svd)])


# Pipe
pipe = Pipeline([('lsi', lsi), ('clf', rfc)])

In [13]:
# Fit
grid_search = GridSearchCV(pipe,params, cv=5, n_jobs=-1, verbose=1)
grid_search.fit(data.data, data.target)

Fitting 5 folds for each of 9 candidates, totalling 45 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  45 out of  45 | elapsed:   23.7s finished
/Users/jonathansokoll/anaconda3/envs/U4-S1-NLP-DS6/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=Pipeline(memory=None,
     steps=[('lsi', Pipeline(memory=None,
     steps=[('vect', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm=...obs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False))]),
       fit_params=None, iid='warn', n_jobs=4,
       param_grid={'lsi__svd__n_components': [10, 100, 250], 'lsi__vect__max_df': [0.9, 0.95, 1.0]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=1)

In [14]:
grid_search.best_score_

0.8786464410735122

## Follow Along
1. Join the Kaggle Competition
2. Download the data
3. Train a model & try: 
    - Creating a Text Extraction & Classification Pipeline
    - Tune the pipeline with a `GridSearchCV` or `RandomizedSearchCV`
    - Add some Latent Semantic Indexing (lsi) into your pipeline. *Note:* You can grid search a nested pipeline, but you have to use double underscores ie `lsi__svd__n_components`
4. Make a submission to Kaggle 


### Define Pipeline Components

In [6]:
vect = TfidfVectorizer(stop_words='english', ngram_range=(1,2))
clf = RandomForestClassifier()
svd = TruncatedSVD(algorithm='randomized', n_iter=10)
lsi = Pipeline([('vect', vect), ('svd', svd)])

pipe = Pipeline([('lsi', lsi), ('clf', clf)])

### Define Your Search Space
You're looking for both the best hyperparameters of your vectorizer and your classification model. 

In [13]:
from sklearn.model_selection import RandomizedSearchCV

parameters = {
    'lsi__svd__n_components': [10,50,75],
    'lsi__vect__max_df': (0.75, 1.0),
    'lsi__vect__min_df': (0.01, 0.05),
    'clf__max_depth':(5,10,15,20)
}

search = RandomizedSearchCV(pipe,parameters, n_iter=10, cv=5, n_jobs=-1, verbose=1, return_train_score=True)
search.fit(X_train, y_train)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:  1.4min finished
C:\Anaconda3\envs\Unit4_Sprint1\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomizedSearchCV(cv=5, error_score='raise-deprecating',
          estimator=Pipeline(memory=None,
     steps=[('lsi', Pipeline(memory=None,
     steps=[('vect', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 2), norm=...obs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False))]),
          fit_params=None, iid='warn', n_iter=10, n_jobs=-1,
          param_distributions={'lsi__svd__n_components': [10, 50, 75], 'lsi__vect__max_df': (0.75, 1.0), 'lsi__vect__min_df': (0.01, 0.05), 'clf__max_depth': (5, 10, 15, 20)},
          pre_dispatch='2*n_jobs', random_state=None, refit=True,
          return_train_score=True, scoring=None, verbose=1)

In [14]:
search.best_score_

0.8766434648105181

### Make a Submission File
*Note:* You are only allowed two submissions a day. Only submit if you feel you cannot achieve higher test accuracy. 

In [15]:
# Predictions on test sample
pred = search.predict(X_test)

In [16]:
submission = pd.DataFrame({'id': test['id'], 'category':pred})
submission['category'] = submission['category'].astype('int64')

In [17]:
# Make Sure the Category is an Integer
submission.head()

,id,category
0,955,2
1,3532,3
2,1390,1
3,1024,1
4,1902,1


I forgot to pull the most recent changes so I didn't have the below two cells in my notebook. Copying them here for posterity...

In [ ]:
subNumber = 0

In [ ]:
# Save your Submission File
# Best to Use an Integer or Timestamp for different versions of your model

submission.to_csv(f'./data/submission{subNumber}.csv', index=False)
subNumber += 1

In [18]:
# Save your Submission File
# Best to Use an Integer or Timestamp for different versions of your model
submission.to_csv('./whiskey-submission-3.csv', index=False)

## Challenge

Continue to apply Latent Semantic Indexing (LSI) to various datasets. 

# Word Embeddings with Spacy (Learn)
<a id="p3"></a>

# Overview

In [6]:
import spacy
nlp = spacy.load("en_core_web_lg")

In [20]:
doc = nlp("Two bananas in pyjamas")

In [21]:
bananas_vector = doc.vector
print(len(bananas_vector))

300


In [26]:
len(X) == len(data.data)

True

In [21]:
def get_word_vectors(docs):
    return [nlp(doc).vector for doc in docs]

In [23]:
X = get_word_vectors(data.data)

rfc.fit(X, data.target)

/Users/jonathansokoll/anaconda3/envs/U4-S1-NLP-DS6/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [27]:
rfc.score(X, data.target)

0.9918319719953326

## Follow Along

In [22]:
X_train_vect = get_word_vectors(X_train)

assert len(X_train_vect) == len(X_train)

In [23]:
rfc = RandomForestClassifier()

In [25]:
rfc.fit(X_train_vect, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [27]:
rfc.score(X_train_vect, y_train)

0.9930394431554525

In [28]:
pred = search.predict(X_test)

In [29]:
submission = pd.DataFrame({'id': test['id'], 'category':pred})
submission['category'] = submission['category'].astype('int64')

In [30]:
submission.head()

,id,category
0,955,2
1,3532,3
2,1390,1
3,1024,1
4,1902,1


In [31]:
submission.to_csv('./whiskey-submission-4.csv', index=False)

In [32]:
# majority vote

files = ['whiskey-submission-1.csv', 'whiskey-submission-3.csv', 'whiskey-submission-4.csv']

target = 'category'
submissions = (pd.read_csv(file)[[target]] for file in files)
ensemble = pd.concat(submissions, axis='columns')
majority_vote = ensemble.mode(axis='columns')[0]

submission[target] = majority_vote
submission.to_csv('./whiskey-submission-5.csv', index=False)

## Challenge

What you should be doing now:
1. Join the Kaggle Competition
2. Download the data
3. Train a model & try: 
    - Creating a Text Extraction & Classification Pipeline
    - Tune the pipeline with a `GridSearchCV` or `RandomizedSearchCV`
    - Add some Latent Semantic Indexing (lsi) into your pipeline. *Note:* You can grid search a nested pipeline, but you have to use double underscores ie `lsi__svd__n_components`
    - Try to extract word embeddings with Spacy and use those embeddings as your features for a classification model.
4. Make a submission to Kaggle 

# Review

To review this module: 
* Continue working on the Kaggle comeptition
* Find another text classification task to work on

In [1]:
def get_tokens(docs):
    tokenized = []
    for doc in docs:
        nlp_doc = nlp(doc)
        tokens = ''
        for token in nlp_doc:
            if (token.is_stop is not True) and (token.is_punct is not True):
                tokens = tokens + ' ' + token.text.lower()
        tokenized.append(tokens)
    return tokenized

In [38]:
nlp(X_train[0])[11]

mature

In [31]:
X_train_token = get_tokens(X_train)

In [8]:
vect = TfidfVectorizer(stop_words='english', ngram_range=(1,2))

In [9]:
# logistic regression

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import RandomizedSearchCV

pipe = Pipeline([
    ('vect', vect),
    ('logreg', LogisticRegression())
])

params = {
    'vect__max_df': [0.7, 0.8, 0.9, 1.0],
    'vect__min_df': [0.01, 0.1, 0.15, 0.2],
    'vect__max_features': [100, 250, 500, 1000],
    'logreg__solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']
}

search = RandomizedSearchCV(pipe, params, n_iter=200, cv=10, scoring='accuracy', verbose=10, n_jobs=-1)

search.fit(X_train_token, y_train)

Fitting 10 folds for each of 200 candidates, totalling 2000 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:    3.4s
[Parallel(n_jobs=-1)]: Done   8 tasks      | elapsed:    3.8s
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:    4.8s
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:    5.9s
[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed:    7.1s
[Parallel(n_jobs=-1)]: Done  48 tasks      | elapsed:    8.0s
[Parallel(n_jobs=-1)]: Done  61 tasks      | elapsed:    9.5s
[Parallel(n_jobs=-1)]: Done  74 tasks      | elapsed:   10.8s
[Parallel(n_jobs=-1)]: Done  89 tasks      | elapsed:   12.8s
[Parallel(n_jobs=-1)]: Done 104 tasks      | elapsed:   14.2s
[Parallel(n_jobs=-1)]: Done 121 tasks      | elapsed:   16.0s
[Parallel(n_jobs=-1)]: Done 138 tasks      | elapsed:   17.9s
[Parallel(n_jobs=-1)]: Done 157 tasks      | elapsed:   19.5s
[Parallel(n_jobs=-1)]: Done 176 tasks      | elapsed:   21.5s
[Parallel(n_jobs=-1)]: Done 197 tasks      | elapsed:  

RandomizedSearchCV(cv=10, error_score='raise-deprecating',
          estimator=Pipeline(memory=None,
     steps=[('vect', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 2), norm='l2', preprocessor=None, smooth_idf=True,
...penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False))]),
          fit_params=None, iid='warn', n_iter=200, n_jobs=-1,
          param_distributions={'vect__max_df': [0.7, 0.8, 0.9, 1.0], 'vect__min_df': [0.01, 0.1, 0.15, 0.2], 'vect__max_features': [100, 250, 500, 1000], 'logreg__solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']},
          pre_dispatch='2*n_jobs', random_state=None, refit=True,
          return_train_score='warn', scoring='accuracy', verbose=10)

In [47]:
search.best_score_

0.8982985305491106

In [10]:
search.best_params_

{'vect__min_df': 0.01,
 'vect__max_features': 500,
 'vect__max_df': 0.7,
 'logreg__solver': 'saga'}

In [42]:
def make_submission(search, number):
    pred = search.predict(X_test)
    submission = pd.DataFrame({'id': test['id'], 'category':pred})
    submission['category'] = submission['category'].astype('int64')
    submission.to_csv(f'./whiskey-submission-{number}.csv', index=False)

In [49]:
make_submission(search, 5)

In [50]:
from sklearn.ensemble import RandomForestClassifier
from scipy.stats import randint, uniform

rfc = RandomForestClassifier()

pipe = Pipeline([
    ('vect', vect),
    ('model', rfc)
])

params = {
    'vect__max_df': [0.7, 0.8, 0.9, 1.0],
    'vect__min_df': [0.01, 0.1, 0.15, 0.2],
    'model__n_estimators': randint(50, 500),
    'model__max_depth': [5, 10, 15, 20, None],
    'model__max_features': uniform(0,1)
}

search = RandomizedSearchCV(pipe, params, n_iter=200, cv=10, scoring='accuracy', verbose=10, n_jobs=-1)

search.fit(X_train_token, y_train)

Fitting 10 folds for each of 200 candidates, totalling 2000 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:    6.4s
[Parallel(n_jobs=-1)]: Done   8 tasks      | elapsed:    6.9s
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:   13.8s
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:   23.0s
[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed:   28.9s
[Parallel(n_jobs=-1)]: Done  48 tasks      | elapsed:   33.5s
[Parallel(n_jobs=-1)]: Done  61 tasks      | elapsed:   45.1s
[Parallel(n_jobs=-1)]: Done  74 tasks      | elapsed:   53.9s
[Parallel(n_jobs=-1)]: Done  89 tasks      | elapsed:  1.4min
[Parallel(n_jobs=-1)]: Done 104 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-1)]: Done 121 tasks      | elapsed:  1.7min
[Parallel(n_jobs=-1)]: Done 138 tasks      | elapsed:  1.8min
[Parallel(n_jobs=-1)]: Done 157 tasks      | elapsed:  1.9min
[Parallel(n_jobs=-1)]: Done 176 tasks      | elapsed:  2.0min
[Parallel(n_jobs=-1)]: Done 197 tasks      | elapsed:  

RandomizedSearchCV(cv=10, error_score='raise-deprecating',
          estimator=Pipeline(memory=None,
     steps=[('vect', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 3), norm='l2', preprocessor=None, smooth_idf=True,
...obs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False))]),
          fit_params=None, iid='warn', n_iter=200, n_jobs=-1,
          param_distributions={'vect__max_df': [0.7, 0.8, 0.9, 1.0], 'vect__min_df': [0.01, 0.1, 0.15, 0.2], 'model__n_estimators': <scipy.stats._distn_infrastructure.rv_frozen object at 0x000001E85E209390>, 'model__max_depth': [5, 10, 15, 20, None], 'model__max_features': <scipy.stats._distn_infrastructure.rv_frozen object at 0x000001E85E2092B0>},
          pre_dispatch='2*n_jobs', random_state=None, refit=True,
         

In [51]:
search.best_score_

0.9017788089713844

In [52]:
make_submission(search, 6)

In [11]:
vect = TfidfVectorizer(stop_words='english', ngram_range=(1,2), min_df=0.01, max_df=0.7, max_features=500)

In [14]:
X_train_vect = vect.fit_transform(X_train_token)

In [15]:
from sklearn.model_selection import train_test_split

X_train_vect, X_val_vect = train_test_split(X_train_vect, shuffle=False)

In [16]:
y_train, y_val = train_test_split(y_train, shuffle=False)

In [25]:
from xgboost import XGBClassifier

eval_set = [(X_train_vect, y_train),
            (X_val_vect, y_val)]

model = XGBClassifier(
    n_estimators=1000,
    max_depth=5,
    learning_rate=0.1,
    n_jobs=-1
)

model.fit(X_train_vect, y_train, eval_set=eval_set, eval_metric='merror', early_stopping_rounds=50)

[0]	validation_0-merror:0.103146	validation_1-merror:0.25966
Multiple eval metrics have been passed: 'validation_1-merror' will be used for early stopping.

Will train until validation_1-merror hasn't improved in 50 rounds.
[1]	validation_0-merror:0.098504	validation_1-merror:0.244204
[2]	validation_0-merror:0.098504	validation_1-merror:0.244204
[3]	validation_0-merror:0.096957	validation_1-merror:0.236476
[4]	validation_0-merror:0.092316	validation_1-merror:0.242658
[5]	validation_0-merror:0.0918	validation_1-merror:0.241113
[6]	validation_0-merror:0.089737	validation_1-merror:0.242658
[7]	validation_0-merror:0.087158	validation_1-merror:0.244204
[8]	validation_0-merror:0.08097	validation_1-merror:0.231839
[9]	validation_0-merror:0.082001	validation_1-merror:0.230294
[10]	validation_0-merror:0.078391	validation_1-merror:0.228748
[11]	validation_0-merror:0.079938	validation_1-merror:0.224111
[12]	validation_0-merror:0.073749	validation_1-merror:0.224111
[13]	validation_0-merror:0.07478

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bynode=1, colsample_bytree=1, gamma=0, learning_rate=0.1,
       max_delta_step=0, max_depth=5, min_child_weight=1, missing=None,
       n_estimators=1000, n_jobs=-1, nthread=None,
       objective='multi:softprob', random_state=0, reg_alpha=0,
       reg_lambda=1, scale_pos_weight=1, seed=None, silent=None,
       subsample=1, verbosity=1)

In [32]:
X_train_vect = vect.fit_transform(X_train_token)

In [39]:
y_train = train['category']

In [40]:
model = XGBClassifier(
    n_estimators=1000,
    max_depth=5,
    learning_rate=0.1,
    n_jobs=-1
)

model.fit(X_train_vect, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bynode=1, colsample_bytree=1, gamma=0, learning_rate=0.1,
       max_delta_step=0, max_depth=5, min_child_weight=1, missing=None,
       n_estimators=1000, n_jobs=-1, nthread=None,
       objective='multi:softprob', random_state=0, reg_alpha=0,
       reg_lambda=1, scale_pos_weight=1, seed=None, silent=None,
       subsample=1, verbosity=1)

In [50]:
X_test_token = get_tokens(X_test)

In [52]:
X_test_vect = vect.transform(X_test_token)

In [53]:
pred = model.predict(X_test_vect)

In [54]:
submission = pd.DataFrame({'id': test['id'], 'category':pred})
submission['category'] = submission['category'].astype('int64')
submission.to_csv('./whiskey-submission-7.csv', index=False)

The above got me over 94% accuracy--and even more importantly, let me beat Jeremy!